<a href="https://colab.research.google.com/github/orodriguezq/orodriguezq-Senales_Y_Sistemas/blob/main/SerieyTransformadaFourier/youtube_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#cargar datos desde drive acceso libre
#####https://docs.google.com/spreadsheets/d/1gBOPsFJDSV1tz0NmZsjbyamuYdtHBMsE/edit?usp=sharing&ouid=104855538492041747764&rtpof=true&sd=true
FILEID = "1gBOPsFJDSV1tz0NmZsjbyamuYdtHBMsE"
##"1DxI5wQpqEWksw2BqJnG7n0IgNQ_xByX-"

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O  canciones.xlsx && rm -rf /tmp/cookies.txt
#!unzip -o codigos.zip
!dir

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file_ = 'canciones.xlsx'#leer archivo xlsx con link, band, type
X  = pd.read_excel(file_)
X#imprimir filas iniciales

In [ ]:
#instalar librerias necesarias para descargar audios youtube
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile

import os
import yt_dlp as youtube_dl

#funcion para descargar mp3 desde youtube
def download_ytvid_as_mp3(video_url,name):
    #video_url = input("enter url of youtube video:")
    video_info = youtube_dl.YoutubeDL().extract_info(url = video_url,download=False)
    filename = f"{name}.mp3"
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':filename,
    }

    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])

    print("Download complete... {}".format(filename))

In [ ]:
import subprocess
import os

#crear carpeta con resultados
try:
  os.mkdir('results')
except:
  print("Carpeta results ya existe")

#recorrer excel con videos
N, P = X.shape
Ns = N * 5 #cantidad de segmentos por cancion

for n in range(N):
    print(f"video {n+1} de {N}")
    print(f"link: {X.loc[n,'link']}\n")
    print(f"band: {X.loc[n,'band']}\n")
    print(f"type: {X.loc[n,'type']}\n")
    #ruta video n-th
    name_ = 'results/'+X.loc[n,'band']+"_"+str(n)+"_"+str(X.loc[n,'type_num']) # #video+nombre+tipo de genero musical
    #descargar mp3 desde youtube
    download_ytvid_as_mp3(X.loc[n,'link'],name_)
    #convertir a .wav
    subprocess.call(['ffmpeg','-y', '-i', name_+'.mp3',
                   name_+'.wav'])

In [ ]:
#cargar .wavs y partir audios
#lista archivos .wav
path = 'results/'
wav_files = [f for f in os.listdir(path) if f.endswith('.wav')]
wav_files

In [ ]:
!pip install soundfile #instalar sondfile

In [ ]:
import soundfile as sf # para instalar pip install soundfile
#leer archivos y crear np.array audios
fs = 48000
tl = np.array([40,50,60,70,80,90,100]) #puntos lectura
ts = 5 #t segmento
Ns = len(wav_files)*len(tl) #cantidad segmentos
x_t = np.zeros((Ns,int(ts*fs),2)) #Ns segmentos, cantidad de muestras, 2 canales (stereo)
label = np.zeros((Ns,1)) #vector tipo de genero
name_c = []
#leer archivos wav
i = 0
for name in wav_files:#lectura audio .wav
    x, fs = sf.read(path+name)
    for ti in tl: #segmentos de tiempo
        x_t[i] = x[int(fs*ti):int(fs*(ti+ts)),:]
        label[i] = int(name[-5]) #tipo de genero
        name_c += [name[:-6]]
        print(f"{i} lectura: {name}; segundo {ti}:{ti+ts}; tipo música {label[i]}")
        i+=1
x_t.shape

In [ ]:
from IPython.display import Audio #reproducir segmento
i = 6
Audio(x_t[i].T,rate=fs)

In [ ]:
#calculo de fourier
vf = np.fft.rfftfreq(x_t.shape[1],1/fs) #calculo vector de frecuencias
Xw = np.fft.rfft(x_t,axis=1).mean(axis=-1) #transformada rapida de Fourier para señal Real a lo largo del tiempo (axis=1) y se promedian los dos canales
Xw.shape

In [ ]:
#grafica tiempo y fourier
plt.plot(np.arange(0,ts,1/fs),x_t.mean(axis=-1).T) #se promedian los dos canales stereo
plt.xlabel('t[s]')
plt.ylabel('x(t)')
plt.show()

In [ ]:
plt.plot(vf,abs(Xw).T)
plt.xlabel('f[Hz]')
plt.ylabel('|X(f)|')
plt.show()

In [ ]:
#se normalizan espectros entre 0 y 1 para evitar inconsistencias por ampliltudes máximas
from sklearn.preprocessing import MinMaxScaler
sca = MinMaxScaler()
Xw_ = sca.fit_transform(abs(Xw).T).T

plt.plot(vf,Xw_.T)
plt.xlabel('f[Hz]')
plt.ylabel('|X(f)|')
plt.show()

#en dB
plt.plot(vf,(20*np.log10(Xw_+1e-10)).T) # se suma 1e-10 para evitar discontinuidad del log
plt.xlabel('f[Hz]')
plt.ylabel('|X(f)| dB')
plt.show()

**Nota**: Generalmente el espectro se presenta en [decibeles [dB]](https://es.wikipedia.org/wiki/Decibelio)

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
#visualización de datos
red_ = TSNE(perplexity = 20,n_components=2,random_state=123,learning_rate='auto',init='pca')
#red_ = PCA(n_components=2,random_state=123)
fmax = 7000
X_2D = red_.fit_transform(Xw_[:,:fmax]) #se tiene en cuenta el espectro hasta fmax Hz

In [ ]:
#graficar separabilidad 2D
plt.scatter(X_2D[:,0],X_2D[:,1],c=label)
plt.colorbar()
plt.show()

color_ = ["b","y"]
#nombre cancion
plt.scatter(X_2D[:,0],X_2D[:,1],c=label,s=1)
for i, tex in enumerate(name_c):
    #print(f"{i} {tex}")
    plt.text(X_2D[i,0]*1.025,X_2D[i,1]*1.025, tex[:-2]+"_"+str(i), fontsize=6,color=color_[int(label[i]-1)])

#plt.colorbar()
plt.show()

In [ ]:
#reproducir audio
i = 69
Audio(x_t[i].T,rate=fs)

In [ ]:
i = 14
Audio(x_t[i].T,rate=fs)

In [ ]:
import joblib
#guardar modelo
os.mkdir('modelo')
filename_ = 'modelo/reggaeton_vs_salsa'
model_ ={'Xw_':Xw_,'fmax': fmax, 'label' : label, 'name_c' : name_c, 'vf':vf,'fs':fs}
joblib.dump(model_,filename_+".pkl")


In [ ]:
#descargar modelo
#guardar resultados
from google.colab import files
from datetime import date, datetime
import shutil
#guardar resultados
namefile = str(datetime.now().strftime("%Y_%m_%d_%H_%M_%d"))+'modelo'
shutil.make_archive(namefile, 'zip', 'modelo')
files.download(namefile+'.zip')

#el archivo .zip puede cargarse en drive y utilizarse en otro cuaderno para detectar género musical de nuevos segmentos

In [ ]:
#cargar modelo
my_model_loaded = joblib.load(filename_+".pkl")
my_model_loaded.keys()

In [ ]:
# ——————————————————————————————————————————————————————————————————————————
# 1. DESCARGA DE CANCIONES DESDE EL EXCEL
# ——————————————————————————————————————————————————————————————————————————

# Identificador de tu Google Sheet con las 10 canciones
FILEID = "1gBOPsFJDSV1tz0NmZsjbyamuYdtHBMsE"

# Descargamos el Excel desde Google Drive
!wget --load-cookies /tmp/cookies.txt \
 "https://docs.google.com/uc?export=download&confirm=$(wget --quiet \
  --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate \
  'https://docs.google.com/uc?export=download&id='$FILEID -O- | \
  sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID \
  -O canciones.xlsx && rm -rf /tmp/cookies.txt

import pandas as pd
import numpy as np
import os
import subprocess
import yt_dlp as youtube_dl
import soundfile as sf

# Leemos el Excel que contiene: link, band, type, type_num
X = pd.read_excel("canciones.xlsx")

# Creamos la carpeta donde guardaremos mp3 y wav
os.makedirs('results', exist_ok=True)

# Función para descargar un video de YouTube como MP3
def download_ytvid_as_mp3(video_url, name):
    video_info = youtube_dl.YoutubeDL().extract_info(url=video_url, download=False)
    filename = f"{name}.mp3"
    options = {
        'format': 'bestaudio/best',
        'keepvideo': False,
        'outtmpl': filename,
    }
    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])
    print(f"Download complete: {filename}")

# Iteramos sobre las filas del Excel, descargamos y convertimos a WAV
N = len(X)
for n in range(N):
    link  = X.loc[n, 'link']
    band  = X.loc[n, 'band']
    tnum  = X.loc[n, 'type_num']
    name_ = f"results/{band}_{n}_{tnum}"
    print(f"\n=== Descargando canción {n+1}/{N} ===")
    print(f"Link: {link}")
    download_ytvid_as_mp3(link, name_)
    # Convertimos el MP3 a WAV (48 kHz, estéreo si aplica)
    subprocess.call([
        'ffmpeg', '-y',
        '-i', f"{name_}.mp3",
        '-ar', '48000',   # sample rate 48 kHz
        '-ac', '2',       # 2 canales estéreo
        f"{name_}.wav"
    ])
    # Borramos el mp3 original para ahorrar espacio
    os.remove(f"{name_}.mp3")


# ——————————————————————————————————————————————————————————————————————————
# 2. PARTICIONAR CADA WAV EN 5 FRAGMENTOS DE 5 SEGUNDOS → 50 MUESTRAS TOTALES
# ——————————————————————————————————————————————————————————————————————————

# Listamos todos los archivos .wav que acabamos de generar
path = 'results/'
wav_files = [f for f in os.listdir(path) if f.endswith('.wav')]
print(f"\nTotal de archivos WAV encontrados: {len(wav_files)} (deberían ser 10)")

# Parámetros de segmentación
fs = 48000            # frecuencia de muestreo (48 kHz)
ts = 5                # duración de cada segmento en segundos
# Para obtener 50 segmentos en total a partir de 10 canciones,
# necesitamos 5 offsets (5 segundos de duración) por cada canción:
tl = [10,  40,  70, 100, 130]   # instantes (en segundos) donde empieza cada fragmento de 5 s

Ns = len(wav_files) * len(tl)   # 10 canciones × 5 offsets = 50 segmentos
# Preasignamos arrays:
x_t    = np.zeros((Ns, int(ts*fs), 2), dtype=np.float32)  # Ns segmentos, muestras (5s×48kHz), 2 canales
label  = np.zeros((Ns, 1),  dtype=int)                    # etiqueta de género (1 o 2)
name_c = []                                               # nombre de canción (sin sufijo "_n_type")

i = 0
for name in wav_files:
    filepath = os.path.join(path, name)
    x, fs_file = sf.read(filepath)    # x.shape = (total_muestras, 2)
    if fs_file != fs:
        raise RuntimeError(f"Esperamos 48 kHz, pero {name} está a {fs_file} Hz")
    # Etiqueta de género tomada del penúltimo carácter antes de ".wav": "..._{type}.wav"
    genre_num = int(name[-5])  # asumiendo formato "..._{type}.wav"
    base_name = name[:-6]      # para evitar "..._n_t.wav"
    for ti_seg in tl:
        start = int(ti_seg * fs)
        end   = int((ti_seg + ts) * fs)
        x_t[i]   = x[start:end, :]      # recortamos 5 segundos (estéreo)
        label[i] = genre_num
        name_c.append(base_name)
        print(f"{i:02d} → {name}   segundos {ti_seg}:{ti_seg+ts}   género {genre_num}")
        i += 1

print(f"\nForma de x_t: {x_t.shape}   → 50 segmentos de 5 s en estéreo")
print(f"Forma de label: {label.shape}")

# ——————————————————————————————————————————————————————————————————————————
# 3. CÁLCULO DE LA FFT Y NORMALIZACIÓN DE CADA SEGMENTO
# ——————————————————————————————————————————————————————————————————————————

# 3.A. Construimos el vector de frecuencias completo para rfft (para ts×fs muestras)
vf = np.fft.rfftfreq(int(ts*fs), 1/fs)

# 3.B. Calculamos la FFT real para cada segmento (axis=1 recorre el tiempo)
#      Luego promediamos ambos canales en el dominio de magnitudes.
Xw = np.fft.rfft(x_t, axis=1)          # shape = (50, N_bins, 2)
Xw_mag = np.mean(np.abs(Xw), axis=2)   # shape = (50, N_bins)

# 3.C. Escogemos un fmax razonable (p. ej. 7000 Hz) para recortar el espectro
fmax = 7000
fmax_idx = np.argmin(np.abs(vf - fmax))  # índice del bin más cercano a fmax
print(f"\nBin máximo seleccionado (f <= {fmax} Hz):  {fmax_idx}  (f = {vf[fmax_idx]:.1f} Hz)")

# 3.D. Cortamos el espectro a [0 : fmax_idx]
Xw_crop = Xw_mag[:, :fmax_idx]

# 3.E. Normalizamos entre 0 y 1 cada vector de frecuencias
from sklearn.preprocessing import MinMaxScaler
sca = MinMaxScaler()
Xw_norm = sca.fit_transform(Xw_crop)   # shape = (50, fmax_idx)

print(f"Dataset final de espectros normalizados:  {Xw_norm.shape}   (50 muestras × {fmax_idx} bins)")

# ——————————————————————————————————————————————————————————————————————————
# 4. VISUALIZACIÓN RÁPIDA Y SEPARABILIDAD 2D
# ——————————————————————————————————————————————————————————————————————————

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# 4.A. Graficar magnitud del primer segmento (solo para verificar)
plt.figure(figsize=(6,3))
plt.plot(vf[:fmax_idx], Xw_norm[0], color='tab:blue')
plt.title("Espectro normalizado (fragmento #0)")
plt.xlabel("Frecuencia [Hz]")
plt.ylabel("Magnitud normalizada")
plt.grid(True)
plt.show()

# 4.B. Separabilidad 2D usando TSNE
red_ = TSNE(perplexity=20, n_components=2, random_state=123, learning_rate='auto', init='pca')
X_2D = red_.fit_transform(Xw_norm)  # shape = (50, 2)

# Graficar scatter con colorbar
plt.figure(figsize=(5,4))
plt.scatter(X_2D[:,0], X_2D[:,1], c=label.ravel(), cmap='viridis', s=20)
plt.colorbar(label="Género (1 vs 2)")
plt.title("Separabilidad 2D (TSNE) de los 50 segmentos")
plt.xlabel("Dim 1")
plt.ylabel("Dim 2")
plt.grid(True)
plt.show()

# Graficar scatter con nombres de canciones
color_ = ["b", "y"]
plt.figure(figsize=(5,4))
plt.scatter(X_2D[:,0], X_2D[:,1], c=label.ravel(), s=10, cmap='viridis')
for idx, tex in enumerate(name_c):
    plt.text(
        X_2D[idx,0] * 1.025,
        X_2D[idx,1] * 1.025,
        tex.split('/')[-1],
        fontsize=6,
        color=color_[int(label[idx,0] - 1)]
    )
plt.title("Mapa 2D con etiquetas de canciones")
plt.xlabel("Dim 1")
plt.ylabel("Dim 2")
plt.grid(True)
plt.show()

# ——————————————————————————————————————————————————————————————————————————
# 5. ENTRENAMIENTO DEL CLASIFICADOR KNN (DISTANCIA EUCLÍDEA)
# ——————————————————————————————————————————————————————————————————————————

from sklearn.neighbors import KNeighborsClassifier
import joblib

knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn.fit(Xw_norm, label.ravel())

os.makedirs('modelo', exist_ok=True)
joblib.dump({
    'knn_model': knn,
    'scaler'   : sca,
    'fmax_idx' : fmax_idx,
    'vf'       : vf[:fmax_idx],
    'fs'       : fs
}, 'modelo/knn_genre_detector.pkl')

print("\n—> Entrenamiento completado: 50 vectores (5 s cada uno) con KNN guardado en modelo/knn_genre_detector.pkl\n")

# ——————————————————————————————————————————————————————————————————————————
# 6. CREAR UN FRAGMENTO DE PRUEBA EXTRAÍDO DE LOS 50 SEGMENTOS Y GUARDARLO COMO WAV
# ——————————————————————————————————————————————————————————————————————————

i_test = 32
test_fragment = x_t[i_test]   # forma (240000, 2)
sf.write("nuevo_fragmento_5s.wav", test_fragment, fs)
print(f"→ Se creó 'nuevo_fragmento_5s.wav' a partir del segmento #{i_test} (género real = {int(label[i_test,0])})\n")

# ——————————————————————————————————————————————————————————————————————————
# 7. FUNCIÓN PARA DETECTAR GÉNERO DE UN NUEVO FRAGMENTO DE 5 s
# ——————————————————————————————————————————————————————————————————————————

def predict_genre_from_wav(new_wav_path: str, model_path: str = 'modelo/knn_genre_detector.pkl'):
    """
    Carga un archivo WAV de exactamente 5 segundos (mono o estéreo),
    calcula su FFT, recorta hasta fmax_idx, normaliza con el mismo MinMaxScaler
    y predice el género usando el KNN entrenado.
    Devuelve la etiqueta (1 o 2).
    """
    mdl = joblib.load(model_path)
    knn_loaded = mdl['knn_model']
    sca_loaded = mdl['scaler']
    fmax_idx  = mdl['fmax_idx']
    fs_ref    = mdl['fs']

    x5, fs_file = sf.read(new_wav_path)
    if fs_file != fs_ref:
        raise ValueError(f"El WAV está a {fs_file} Hz; debe estar a {fs_ref} Hz.")
    n5 = 5 * fs_ref
    if x5.ndim == 1:
        if x5.shape[0] < n5:
            raise ValueError("El archivo WAV es más corto de 5 segundos.")
        x5_seg = x5[:n5]
    else:
        if x5.shape[0] < n5:
            raise ValueError("El WAV (estéreo) es más corto de 5 segundos.")
        x5_seg = x5[:n5, :]

    if x5_seg.ndim == 1:
        Xw5 = np.abs(np.fft.rfft(x5_seg))
    else:
        Xw5_0 = np.abs(np.fft.rfft(x5_seg[:, 0]))
        Xw5_1 = np.abs(np.fft.rfft(x5_seg[:, 1]))
        Xw5   = (Xw5_0 + Xw5_1) / 2.0

    mag5_crop = Xw5[:fmax_idx].reshape(1, -1)
    mag5_norm = sca_loaded.transform(mag5_crop)

    pred = knn_loaded.predict(mag5_norm)[0]
    return int(pred)

# ——————————————————————————————————————————————————————————————————————————
# 8. EJEMPLO DE USO DEL DETECTOR (SE PREDICE SOBRE 'nuevo_fragmento_5s.wav')
# ——————————————————————————————————————————————————————————————————————————

nuevo_fragmento = "nuevo_fragmento_5s.wav"  # creado en el paso anterior

try:
    genero_predicho = predict_genre_from_wav(nuevo_fragmento)
    print(f"\n¡Género predicho para '{nuevo_fragmento}': {genero_predicho}  (2=Salsa, 1=Reggaetón)!\n")
    display(Audio(nuevo_fragmento, rate=fs))
except Exception as e:
    print("Error al predecir género:", e)
